<p style="border: 1px solid #e7692c; border-left: 15px solid #e7692c; padding: 10px; text-align:justify;">
    <strong style="color: #e7692c">Tip.</strong> <a style="color: #000000;" href="https://nbviewer.jupyter.org/github/PacktPublishing/Hands-On-Computer-Vision-with-TensorFlow-2/blob/master/Chapter09/ch9_nb3_train_model.ipynb" title="View with Jupyter Online">Click here to view this notebook on <code>nbviewer.jupyter.org</code></a>. 
    <br/>These notebooks are better read there, as Github default viewer ignores some of the formatting and interactive content.
    </p>

<table style="font-size: 1em; padding: 0; margin: 0;">
    <tr style="vertical-align: top; padding: 0; margin: 0;background-color: #ffffff">
        <td style="vertical-align: top; padding: 0; margin: 0; padding-right: 15px;">
    <p style="background: #363636; color:#ffffff; text-align:justify; padding: 10px 25px;">
        <strong style="font-size: 1.0em;"><span style="font-size: 1.2em;"><span style="color: #e7692c;">Hands-on</span> Computer Vision with TensorFlow 2</span><br/>by <em>Eliot Andres</em> & <em>Benjamin Planche</em> (Packt Pub.)</strong><br/><br/>
        <strong>> Chapter 9: Performance and running on mobile</strong><br/>
    </p>

<h1 style="width: 100%; text-align: left; padding: 0px 25px;"><small style="color: #e7692c;">Notebook 3:</small><br/>Training a model and converting it for mobile devices</h1>
<br/>
<p style="border-left: 15px solid #363636; text-align:justify; padding: 0 10px;">
    In this chapter, we covered how to convert and run a model on mobile.
<br/><br/>
    This notebooks trains a model to recognize face expressions and converts it to CoreML, TFLite and TensorFlow.js
</p>
<br/>

<p style="border-left: 15px solid #363636; text-align:justify; padding: 0 10px;">
    <strong> Requirements </strong>
<br/><br/>
    To run this notebook, you need to download the <a href="https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/data">FER dataset</a> and extract it. When done, change the `BASE_PATH` variable to point to the dataset folder.
</p>
<br/>
<p style="border-left: 15px solid #e7692c; padding: 0 10px; text-align:justify;">
    <strong style="color: #e7692c;">Tip.</strong> The notebooks shared on this git repository illustrate some notions from the book "<em><strong>Hands-on Computer Vision with TensorFlow 2</strong></em>" written by Eliot Andres and Benjamin Planche, published by Packt. If you enjoyed the insights shared here, <a href="https://www.amazon.com/Hands-Computer-Vision-TensorFlow-processing/dp/1788830644" title="Learn more about the book!"><strong>please consider acquiring the book!</strong></a>
<br/><br/>
The book provides further guidance for those eager to learn about computer vision and to harness the power of TensorFlow 2 and Keras to build efficient recognition systems for object detection, segmentation, video processing, smartphone applications, and more.</p>
        </td>
        <td style="vertical-align: top; padding: 0; margin: 0; width: 280px;">
    <a href="https://www.amazon.com/Hands-Computer-Vision-TensorFlow-processing/dp/1788830644" title="Learn more about the book!" target="_blank">
        <img src="../banner_images/book_cover.png" width=280>
    </a>
    <p style="background: #e7692c; color:#ffffff; padding: 10px; text-align:justify;"><strong>Leverage deep learning to create powerful image processing apps with TensorFlow 2 and Keras. <br/></strong>Get the book for more insights!</p>
    <ul style="height: 32px; white-space: nowrap; text-align: center; margin: 0px; padding: 0px; padding-top: 10px;">
    <li style="display: block;height: 100%;float: left;vertical-align: middle;margin: 0 25px 10px;padding: 0px;">
        <a href="https://www.amazon.com/Hands-Computer-Vision-TensorFlow-processing/dp/1788830644" title="Get the book on Amazon (paperback or Kindle version)!" target="_blank">
        <img style="vertical-align: middle; max-width: 72px; max-height: 32px;" src="../banner_images/logo_amazon.png" width="75px">
        </a>
    </li>
    <li style="display: inline-block;height: 100%;vertical-align: middle;float: right;margin: -5px 25px 10px;padding: 0px;">
        <a href="https://www.packtpub.com/application-development/hands-computer-vision-tensorflow-2" title="Get your Packt book (paperback, PDF, ePUB, or MOBI version)!" target="_blank">
        <img style="vertical-align: middle; max-width: 72px; max-height: 32px;" src="../banner_images/logo_packt.png" width="75px">
        </a>
    </li>
    </ul>
        </td>
        </tr>
        </table>

In [1]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.layers import Input
from sklearn.model_selection import train_test_split
import pandas as pd
import cv2
import numpy as np
import tensorflow as tf

tf.__version__

'2.5.0'

## Set constants and parameters


In [2]:
# Download data here: https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/data
DATASET_PATH = '../data/fer2013/fer2013.csv'

IMAGE_SIZE = (48, 48)
INPUT_SHAPE = IMAGE_SIZE + (1,)
EMOTIONS = ["angry", "disgust", "scared",
            "happy", "sad", "surprised", "neutral"]
CALLBACK_PATIENCE = 50
BATCH_SIZE = 32
NUM_EPOCHS = 50
VALIDATION_SPLIT = .2
NUM_CLASSES = len(EMOTIONS)
L2_REGULARIZATION = 0.01

## Build the model

In [3]:
input_tensor = Input(shape=INPUT_SHAPE)
model = MobileNet(input_tensor=input_tensor, alpha=1.0,
                    include_top=False, weights=None)

output = tf.keras.layers.Reshape((1024,))(model.output)
output = tf.keras.layers.Dense(7, activation='softmax')(output)
model = tf.keras.Model(model.input, output)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 1)]       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 24, 24, 32)        288       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 24, 24, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 24, 24, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 24, 24, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 24, 24, 32)        128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 24, 24, 32)        0     

## Load data

In [4]:
def load_fer2013():
    data = pd.read_csv(DATASET_PATH)
    pixels = data['pixels'].tolist()
    width, height = 48, 48
    faces = []
    for pixel_sequence in pixels:
        face = [int(pixel) for pixel in pixel_sequence.split(' ')]
        face = np.asarray(face).reshape(width, height)
        face = cv2.resize(face.astype('uint8'), IMAGE_SIZE)
        faces.append(face.astype('float32'))
    faces = np.asarray(faces)
    faces = np.expand_dims(faces, -1)
    emotions = pd.get_dummies(data['emotion']).values
    return faces, emotions

def preprocess_input(x, v2=True):
    x = x.astype('float32')
    x = x / 255.0
    x = x - 0.5
    x = x * 2.0
    return x

data_generator = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=.1,
    horizontal_flip=True)

# loading dataset
faces, emotions = load_fer2013()
faces = preprocess_input(faces)


## Train model

In [5]:
regularization = tf.keras.regularizers.l2(L2_REGULARIZATION)

early_stop = EarlyStopping('val_loss', patience=CALLBACK_PATIENCE)
reduce_lr = ReduceLROnPlateau(
    'val_loss', factor=0.1, patience=int(CALLBACK_PATIENCE/4), verbose=1)
tensorboard = TensorBoard('./logs')
callbacks = [early_stop, reduce_lr, tensorboard]

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
xtrain, xtest, ytrain, ytest = train_test_split(faces, emotions, test_size=0.2)

model.fit_generator(data_generator.flow(xtrain, ytrain, BATCH_SIZE),
                    steps_per_epoch=len(xtrain) / BATCH_SIZE,
                    epochs=NUM_EPOCHS, verbose=1, callbacks=callbacks,
                    validation_data=(xtest, ytest))

/home/free/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
897/897 [==============================] - 161s 178ms/step - loss: 1.8797 - accuracy: 0.2474 - val_loss: 1.8215 - val_accuracy: 0.2598
Epoch 2/50
897/897 [==============================] - 163s 181ms/step - loss: 1.8030 - accuracy: 0.2805 - val_loss: 2.1626 - val_accuracy: 0.2607
Epoch 3/50
897/897 [==============================] - 164s 183ms/step - loss: 1.7437 - accuracy: 0.3145 - val_loss: 1.6619 - val_accuracy: 0.3583
Epoch 4/50
897/897 [==============================] - 168s 187ms/step - loss: 1.7047 - accuracy: 0.3368 - val_loss: 2.0152 - val_accuracy: 0.2763
Epoch 5/50
897/897 [==============================] - 161s 180ms/step - loss: 1.6771 - accuracy: 0.3494 - val_loss: 1.8689 - val_accuracy: 0.2711
Epoch 6/50
897/897 [==============================] - 159s 177ms/step - loss: 1.6520 - accuracy: 0.3553 - val_loss: 1.5316 - val_accuracy: 0.4030
Epoch 7/50
897/897 [==============================] - 154s 172ms/step - loss: 1.5612 - accuracy: 0.3974 - val_loss: 1.4791 -

## Convert to CoreML for IOS

In [8]:
!python3 -m pip install tfcoreml

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 44 kB 1.5 MB/s eta 0:00:011
     |████████████████████████████████| 109.2 MB 8.7 MB/s eta 0:00:01
     |████████████████████████████████| 3.4 MB 7.1 MB/s eta 0:00:01
     |████████████████████████████████| 6.1 MB 7.5 MB/s eta 0:00:01
     |████████████████████████████████| 3.1 MB 7.0 MB/s eta 0:00:01
     |████████████████████████████████| 488 kB 13.3 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 2.5 MB/s eta 0:00:011
     |████████████████████████████████| 532 kB 4.7 MB/s eta 0:00:01
  Created wheel for attr: filename=attr-0.3.1-py3-none-any.whl size=2457 sha256=87f06879244f08cbe1f4985f1d4806fca8f50a0b8cbd9e8aa607d0da6cbe8948
  Stored in directory: /home/free/.cache/pip/wheels/da/51/18/7b8531558401f15a9eebc9aef5a66c59fb7b1d4b84a458ecdf
Successfully built attr
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estim

In [17]:
import tfcoreml as tf_converter
from tensorflow.python.keras.callbacks import TensorBoard
from tensorflow.python.saved_model import tag_constants
from tensorflow.python.tools import freeze_graph
input_saved_model_dir = "./saved_model"

# previous version
# tf.keras.experimental.export_saved_model(
#     model, saved_model_path=input_saved_model_dir, serving_only=False)
tf.saved_model.save(
    model, input_saved_model_dir, signatures=None, options=None
)

## save model with tensorflow v1

In [18]:
output_node_name = 'dense/Softmax'
input_binary = False
input_saver_def_path = False
restore_op_name = None
filename_tensor_name = None
clear_devices = True
input_meta_graph = False
checkpoint_path = None
input_graph_filename = None
saved_model_tags = tag_constants.SERVING


freeze_graph.freeze_graph(input_graph_filename, input_saver_def_path,
                          input_binary, checkpoint_path, output_node_name,
                          restore_op_name, filename_tensor_name,
                          'frozen_model.pb', clear_devices, "", "", "",
                          input_meta_graph, input_saved_model_dir,
                          saved_model_tags)


tf_converter.convert('frozen_model.pb',
                     'mobilenet.mlmodel',
                     class_labels=EMOTIONS,
                     image_input_names=['input_1:0'],
                     output_feature_names=[output_node_name + ':0'],
                     red_bias=-1,
                     green_bias=-1,
                     blue_bias=-1,
                     image_scale=1/127.5,
                     is_bgr=False)

InvalidArgumentError: Attempting to add a duplicate function with name: __inference_conv_pw_4_layer_call_and_return_conditional_losses_165349 where the previous and current definitions differ. Previous definition: signature {
  name: "__inference_conv_pw_4_layer_call_and_return_conditional_losses_165349"
  input_arg {
    name: "inputs"
    type: DT_FLOAT
  }
  input_arg {
    name: "conv2d_readvariableop_resource"
    type: DT_RESOURCE
  }
  output_arg {
    name: "identity"
    type: DT_FLOAT
  }
  is_stateful: true
  control_output: "Conv2D/ReadVariableOp"
}
node_def {
  name: "Conv2D/ReadVariableOp"
  op: "ReadVariableOp"
  input: "conv2d_readvariableop_resource"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  experimental_debug_info {
    original_node_names: "Conv2D/ReadVariableOp"
  }
}
node_def {
  name: "Conv2D"
  op: "Conv2D"
  input: "inputs"
  input: "Conv2D/ReadVariableOp:value:0"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
  att

## Convert to TFLite

In [19]:

converter = tf.lite.TFLiteConverter.from_keras_model(model)
## Or from a SavedModel
# converter = tf.lite.TFLiteConverter('./saved_model')

tflite_model = converter.convert()
open("result.tflite", "wb").write(tflite_model)



12827168

## Convert to TFJS

In [11]:
!python3 -m pip install tensorflowjs

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 64 kB 1.6 MB/s eta 0:00:011
  Using cached tensorflow-2.5.0-cp36-cp36m-manylinux2010_x86_64.whl (454.3 MB)
     |████████████████████████████████| 108 kB 2.4 MB/s eta 0:00:01
  Using cached tensorflow_estimator-2.5.0-py2.py3-none-any.whl (462 kB)
  Using cached tensorboard-2.5.0-py3-none-any.whl (6.0 MB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
      Successfully uninstalled tensorflow-estimator-1.14.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 1.14.0
    Uninstalling tensorflow-1.14.0:
      Successfully uninstalled tensorflow-1.14.0
ERROR: pip's depend

In [20]:
# Convert in the current environment
import sys
!tensorflowjs_converter --input_format=tf_saved_model saved_model my-tfjs --output_format tfjs_graph_model

Writing weight file my-tfjs/model.json...
